In [21]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [22]:
post = pd.read_csv('lv2-2402.csv')

# 1

## 1-1

In [23]:
yes_map = {
    'heart emoji': 5,
    'thumbs up': 4,
    'clap': 3, 
    'laughing': 2,
    np.nan: 1,
    'fire': 0
}

In [24]:
post.loc[post['LIKES'] == 'yes', 'LIKE_SCORE'] = \
    post.loc[post['LIKES'] == 'yes', 'LIKE_TYPE'].map(yes_map)

In [25]:
no_map = {
    'fire': -5,
    np.nan: -1
}

In [26]:
post.loc[post['LIKES'] == 'no', 'LIKE_SCORE'] = \
    post.loc[post['LIKES'] == 'no', 'LIKE_TYPE'].map(no_map).fillna(value=0)

In [20]:
post['LIKE_SCORE'].isna().sum()

0

## 1-2

In [28]:
df_temp = post[post['POST_ID'] == 1]

In [31]:
(df_temp['LIKES'] == 'yes').sum()

13

In [33]:
df_temp['USER_ID'].nunique()

25

In [34]:
df_temp['LIKE_SCORE'].sum()

0.0

In [40]:
AGG24 = post.groupby('POST_ID')\
            .apply(lambda x: pd.Series({'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
                                        'NUM_USER': x['USER_ID'].nunique(),
                                        'SUM_SCORE': x['LIKE_SCORE'].sum()}))
AGG24.head()

,NUM_LIKES,NUM_USER,SUM_SCORE
POST_ID,,,
1,13.0,25.0,0.0
2,15.0,31.0,5.0
3,14.0,27.0,23.0
4,16.0,32.0,22.0
5,13.0,27.0,18.0


## 1-3

In [42]:
AGG24['POPULAR'] = AGG24['NUM_LIKES'] + (AGG24['SUM_SCORE'] / AGG24['NUM_USER'])

In [44]:
round(AGG24['POPULAR'].max(), 2)

20.82

# 2

## 2-1

In [46]:
df = post[post['USER_ID'] == 2]

In [49]:
df['POST_ID'].nunique()

66

In [53]:
df.loc[df['LIKES'] == 'yes', 'POST_ID'].nunique()

29

In [56]:
AGG25 = post.groupby('USER_ID')\
            .apply(lambda x: pd.Series({
                    'NUM_POST': x['POST_ID'].nunique(),
                    'NUM_LIKE': x.loc[x['LIKES'] == 'yes', 'POST_ID'].nunique()}))

In [57]:
AGG25.shape

(77, 2)

## 2-2

In [58]:
from sklearn.preprocessing import StandardScaler

In [65]:
col = [f'{col}_S' for col in AGG25.columns]
col

['NUM_POST_S', 'NUM_LIKE_S']

In [68]:
df_scaled = pd.DataFrame(StandardScaler().fit_transform(AGG25), columns=col)

## 2-3

In [69]:
from sklearn.cluster import KMeans

In [70]:
model = KMeans(n_clusters=2, random_state=1234).fit(df_scaled)

In [72]:
AGG25['cluster'] = model.predict(df_scaled)

In [73]:
AGG25.head()

,NUM_POST,NUM_LIKE,cluster
USER_ID,,,
2,66,29,0
3,67,25,0
4,64,32,0
5,257,135,1
6,62,31,0


## 2-4

In [74]:
AGG25['cluster'].value_counts()

0    64
1    13
Name: cluster, dtype: int64

In [75]:
Cluster1 = 1
Cluster2 = 0

## 2-5

In [81]:
df2 = pd.merge(post, AGG25.reset_index(), on='USER_ID', how='left')

In [86]:
Avg_A = df2[df2['cluster'] == Cluster1]['NUM_HASHTAG'].mean()
Avg_A

2.402274768033523

## 2-6

In [87]:
Avg_B = df2[df2['cluster'] == Cluster2]['NUM_HASHTAG'].mean()
Avg_B

2.428261393778635

In [89]:
round(abs(Avg_A - Avg_B), 3)

0.026

# 3

## 3-1

In [92]:
AGG26 = post.groupby(['POST_ID', 'CREATED_DATE'])\
            .apply(lambda x: pd.Series({'NUM_USER': x['USER_ID'].nunique(),
                                        'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
                                        'AVG_HASHTAG': x['NUM_HASHTAG'].mean()}))\
            .reset_index()

In [93]:
AGG26.shape

(257, 5)

## 3-2

In [95]:
AGG26['POPULAR'] = np.where(AGG26['NUM_USER'] >= 31, 1, 0)

## 3-3

In [97]:
from datetime import datetime

In [101]:
AGG26['WEEKDAY']\
    = AGG26['CREATED_DATE']\
            .map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

## 3-4

In [102]:
DataSet6 = AGG26.copy()

In [103]:
from sklearn.naive_bayes import GaussianNB

In [104]:
col = ['NUM_LIKES', 'AVG_HASHTAG', 'WEEKDAY']

In [105]:
model = GaussianNB().fit(X=DataSet6[col], y=DataSet6['POPULAR'])

## 3-5

In [107]:
test = pd.DataFrame([{'NUM_LIKES': 16, 'AVG_HASHTAG': 2.42, 'WEEKDAY': 5}])
test

,NUM_LIKES,AVG_HASHTAG,WEEKDAY
0,16,2.42,5


In [109]:
DataSet6['POPULAR'].unique()

array([0, 1], dtype=int64)

In [108]:
model.predict_proba(test)

array([[0.22387533, 0.77612467]])

In [110]:
round(0.77612467, 3)

0.776

## 참고

In [111]:
test = np.array([
    [0.1, 0.83, 0.07],
    [0.2, 0.68, 0.12],
    [0.3, 0.1, 0.6]
])
test

array([[0.1 , 0.83, 0.07],
       [0.2 , 0.68, 0.12],
       [0.3 , 0.1 , 0.6 ]])

In [114]:
test[:, 0]

array([0.1, 0.2, 0.3])